In [ ]:
# %load_ext autoreload
# %autoreload 2

import os
os.environ["OMP_NUM_THREADS"] ="1" # export OMP_NUM_THREADS=1
os.environ["OPENBLAS64_NUM_THREADS"] ="1" # export OPENBLAS_NUM_THREADS=1
os.environ["MKL_NUM_THREADS"] ="1" # export MKL_NUM_THREADS=1
os.environ["VECLIB_MAXIMUM_THREADS"] ="1" # export VECLIB_MAXIMUM_THREADS=1
os.environ["NUMEXPR_NUM_THREADS"] ="1" # export NUMEXPR_NUM_THREADS=1

'''Set to run on a single core'''


from  astropy.io import fits

import numpy as np
import scipy as sp

import ppxf.sps_util as ppxf_lib
from ppxf_sew import ewfit
from ppxf_sew import ew_util as util
from ppxf import ppxf_util as ppxf_util

from itertools import chain
from functools import partial
from seaborn import kdeplot
from multiprocessing.pool import Pool
import seaborn as sns
import warnings
from matplotlib import colors
warnings.filterwarnings('ignore')
from urllib import request
from ppxf_sew import all_temp_make
from tqdm import trange
from scipy.interpolate import interp1d

import matplotlib as mpl
mpl.rcParams.update (mpl.rcParamsDefault)
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['lines.color']='black'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 20
mpl.rcParams['font.family'] = 'DejaVu Serif'
mpl.rcParams['font.style'] = "normal"
mpl.rcParams["mathtext.fontset"] = "stix"
mpl.rcParams['legend.framealpha']=1
mpl.rcParams['xtick.major.width']=2
mpl.rcParams['ytick.major.width']=2
mpl.rcParams['legend.numpoints']=1
mpl.rcParams['xtick.labelsize']=15
mpl.rcParams['ytick.labelsize']=15
mpl.rcParams['legend.fontsize']=20
mpl.rcParams['legend.numpoints']=1
mpl.rcParams['xtick.direction']='in'
mpl.rcParams['ytick.direction']='in'

import matplotlib.ticker as ticker
tl=ticker.MaxNLocator(nbins=5)

from matplotlib.pyplot import MultipleLocator

x_major_locator=MultipleLocator(0.2)
y_major_locator=MultipleLocator(0.2)
import matplotlib.pyplot as plt

c = 299792.458

basic function

In [ ]:

def losvd_rfft(pars, nl, ncomp):
    """

    Analytic Fourier Transform (of real input) of the Gauss-Hermite LOSVD.
    Equation (38) of `Cappellari (2017)
    <https://ui.adsabs.harvard.edu/abs/2017MNRAS.466..798C>`_

    """
    losvd_rfft = np.empty((nl, ncomp), dtype=complex)
    p = 0
    for j in range(ncomp):  # loop over kinematic components
        vel, sig = pars[0 + p], pars[1 + p]
        a = vel/sig
        w = np.linspace(0, np.pi*sig, nl)
        losvd_rfft[:, j] = np.exp(1j*a*w - 0.5*w**2)
        p+=2

    return np.conj(losvd_rfft)

ext = lambda x,delta:(x/5500)**-delta

def att_fun(lam,ebv,delta,f_nodust=0,resp=None):
    # np.random.seed(j)
    if resp == 'sti':
        resp_fun = lambda lam: -0.05*(np.sign(lam-5100)-1)
    elif resp == 'cal':
        # resp = lambda lam: 0.1*(1-np.exp(-((lam-5500)/1500)**2))
        resp_fun = lambda lam: 0.05*np.sin(2*np.pi*(lam-5500)/2000)
    elif resp is None:
        resp_fun = lambda lam: 0
    # delta=-np.log(1+1/Rv)/np.log(89/110)
    Rv=1/(((89/110)**-delta)-1)
    # k=(110.-89.**2/110./Rv)/21.
    # ext = lambda x:k*(x/5500)**-1+(1-k)*(x/5500)**-2
    frac = 10**(-0.4*(ebv*Rv*ext(lam,delta).clip(0)))
    frac = f_nodust + (1-f_nodust)*frac
    if resp is None:
        resp=lambda x: 0
    return frac *10**(-0.4*resp_fun(lam))


power =lambda x: (x/5500)**-1

def mock_file(temp_file,lam_range,weight,comp,params,gas,ebv,ebv1,SN,v_delta, delta=1.3,nn=30 ,savefits=False, AGN=0, resp=None):
    
    hdu=fits.open(os.path.join(os.getcwd(),'test_data/spSpec-54592-2974-262.fit'))
    header=hdu[0].header
    header['Z']=0
    
    velscale=c* v_delta*np.log(10)

    if temp_file is None:
        sps_name = 'galaxev'

        basename = f"spectra_{sps_name}_9.0.npz"
        temp_file = os.path.join(os.getcwd(),'sps_models/' + basename)

        FWHM_gal=2.76
        sps = ppxf_lib.sps_lib(temp_file, velscale, FWHM_gal, wave_range=[3000,10000])

        temp_star0= sps.templates.reshape(sps.templates.shape[0], -1)
        lam_temp=np.exp(sps.ln_lam_temp)
    else:

        sps=util.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True)
        temp_star0=sps.templates.reshape(sps.templates.shape[0],-1)

        lam_temp=np.exp(sps.log_lam_temp)

    temp_gas, gas_names, l_wave = util.emission_lines(np.log(lam_temp), 2.76, lam_range, comp[gas], params, tie_balmer=False, limit_doublets=False)   
 
  
    # temp = np.column_stack([temp, gas_templates])
    if AGN>0:
        temp_star0=np.insert(temp_star0, temp_star0.shape[1], power(lam_temp),  axis=1)
        weight=np.insert(weight, temp_star0.shape[1]-1, AGN,  axis=0)
        weight[:temp_star0.shape[1]]/=np.sum(weight[:temp_star0.shape[1]])
        comp=np.insert(comp, temp_star0.shape[1]-1, True)
        gas=np.insert(gas, temp_star0.shape[1]-1, False)


    npad = 2**int(np.ceil(np.log2(temp_star0.shape[0])))  
    templates_rfft = np.fft.rfft(temp_star0, npad, axis=0)
    lvd_rfft=losvd_rfft(params/velscale,templates_rfft.shape[0],3)
    temp_star=np.array([np.fft.irfft(templates_rfft.T[i]*lvd_rfft[:, comp[~gas][i]], npad)[:lam_temp.shape[0]] for i in range(temp_star0.shape[1])]).T
    temp=np.hstack([temp_star,temp_gas])
    n=temp_star0.shape[1]
    numt=np.where((lam_temp>5400)&(lam_temp<5600))   
    temp[:, :n] /= np.mean(temp[numt].T[:n],1)
    num=(lam_temp>lam_range[0])&(lam_temp<lam_range[1])

    galaxy0=temp@weight

    # galaxy_stellar=(temp[:,:temp_star0.shape[1]]@weight[:temp_star0.shape[1]])*att_fun(lam_temp,ebv,delta,resp=resp)
    galaxy_stellar=(temp[:,:nn]@weight[:nn])*att_fun(lam_temp,ebv,delta,resp=resp)*att_fun(lam_temp,ebv1,delta,resp=None)+(temp[:,nn:temp_star0.shape[1]]@weight[nn:temp_star0.shape[1]])*att_fun(lam_temp,ebv,delta,resp=resp)
    
    galaxy_emm=temp[:,temp_star0.shape[1]:]@weight[temp_star0.shape[1]:]
    # galaxy=galaxy_stellar + weight[temp_star0.shape[1]]*temp[:,temp_star0.shape[1]]*att_fun(lam_temp,ebv,delta)
    galaxy=galaxy_stellar/np.mean(galaxy_stellar[numt])
    galaxy+=galaxy_emm
    galaxy[galaxy<=0]=1e-8
    
    noise=galaxy/SN
    galaxy1= np.random.normal(loc=galaxy,scale=noise,size=galaxy.shape[0]) #galaxy-np.random.normal(loc=0,scale=galaxy/SN,size=galaxy.shape[0]) #
    
    # noise = abs(galaxy1-galaxy)
    # galaxy1= np.random.normal(loc=galaxy,scale=np.sqrt(galaxy)/SN,size=galaxy.shape[0])
    # noise = np.sqrt(galaxy)/SN
    lam=lam_temp[num]
    galaxy1=galaxy1[num]
    galaxy=galaxy[num]
    galaxy0=galaxy0[num]
    noise=noise[num]
    galaxy_emm=galaxy_emm[num]
    # temp=temp[num]
    header['COEFF0']=np.log10(lam[0])

    data=[galaxy1,galaxy0,noise]
    if  savefits:
        hdu_img = fits.PrimaryHDU(data=data, header=header)
        hdu_sci = fits.HDUList([hdu_img])
        hdu_sci.writeto(os.path.join(os.getcwd(),'test_data/mock.fits'), overwrite=True,checksum=True)

    return lam, galaxy, galaxy0, galaxy1, galaxy_emm, lam_temp,temp,velscale,noise

In [ ]:
hdu=fits.open(os.path.join(os.getcwd(),'test_data/spSpec-54592-2974-262.fit'))
header=hdu[0].header
header['Z']=0
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
lam_range=np.array([3300,9000])
v_delta=1e-4
velscale= c*v_delta*np.log(10)


tie_balmer=False
limit_doublets=False
template=util.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True)
temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
comp = np.array([0]*120 + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
gas_templates, gas_names, l_wave = util.emission_lines(np.log(lam_temp), 2.76, lam_range, comp[gas], params, tie_balmer=tie_balmer, limit_doublets=limit_doublets)   

mock a galaxy

In [ ]:
"""
initial a mock galaxy
"""
n_star=temp.shape[1] 

lam_range=np.array([3300,9000])

wave = np.array([3798.983, 3836.479, 3890.158, 3971.202, 4102.899, 4341.691, 4862.691, 6564.632])  # vacuum wavelengths
wave1=np.array([3726.032, 3728.815, 6716.44 , 6730.816, 3967.467, 3868.763,4685.704, 5875.614, 5006.843, 6300.298, 6583.452])
ratios = np.array([ 0.0530, 0.0731, 0.105, 0.159, 0.259, 0.468, 1, 2.86]) # [0.0530, 0.0731, 0.105, 0.159,]
ratios *= wave[-2]/wave  # Account for varying log-sampled pixel size in Angstrom
num0=np.where((wave>lam_range[0])&(wave<lam_range[1]))
num1=np.where((wave1>lam_range[0])&(wave1<lam_range[1]))

weights=np.zeros(n_star+num0[0].shape[0]+num1[0].shape[0])

weights[n_star:n_star+num0[0].shape[0]]=5*ratios[num0]
weights[n_star+num0[0].shape[0]:]=2*np.ones(num1[0].shape[0])#np.random.random(size=11)
# weights[120:]=0
num=np.arange(0,n_star)
np.random.seed(0)
np.random.shuffle(num)
np.random.seed()
weights[num[:10]]=1./10
comp = np.array([0]*n_star + [1]*8 + [2]*11)
gas = np.array(comp) >0
params =np.array([50, 80, -10, 100, 30, 50])
ebv=0.3
k=0.
ebv1=k*ebv
sn=30
delta=1.3
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
# temp_file=None
AGN=0.
resp=None
lam_range=np.array([3300,9000])
lam_mock,galaxy1_mock,galaxy0_mock,galaxy1_sn_mock, galaxy_emm_mock,lam_temp0,temp0,velscale,noise=mock_file(temp_file,lam_range,weights,comp,params, gas,ebv,ebv1,sn,v_delta=v_delta,delta=delta,savefits=True,AGN=AGN,resp=resp)
if AGN>0:
    weights=np.insert(weights,120,AGN)
    weights[:121]/=1+AGN
    



In [ ]:
from ppxf_sew import all_temp_make
name='mock.fits'

file=os.path.join(os.getcwd(),'test_data',name)

c = 299792.458

#data read
hdu=fits.open(file)
velscale=c*v_delta*np.log(10)
galaxy=hdu[0].data[0] #galaxy spectrum
galaxy0=hdu[0].data[1] #galaxy spectrum
noise=hdu[0].data[2] #galaxy noise
Z=hdu[0].header['Z'] # galaxy redshift
s=galaxy.shape[0]
mask=np.zeros(s) #galaxy mask
lam=(10**(hdu[0].header['COEFF0']+v_delta*np.arange(s)))/(1+Z) #galaxy lam

# temp read

from ppxf_sew import ew_util as lib


temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
template=lib.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True) #stellar template

temp=template.templates.reshape(template.templates.shape[0],-1)
lam_temp=np.exp(template.log_lam_temp)
lam_range_gal = np.array([np.min(lam), np.max(lam)])

AGN=0.

n_star=temp.shape[1] 


templates,start,moments,component,gas_component,gas_names=all_temp_make(temp,lam_temp,lam_range_gal)
if AGN>0:
    templates=np.insert(templates,120,power(lam_temp),axis=1)
# np.hstack([templates,np.array([power(lam_temp)]).T])
vlim = np.array([-2900, 2900])  # As 2e3 nonlinear_fit() +900 for 3sigma
lam_range = lam_temp[[0, -1]]/np.exp(vlim/c)
num=(lam > lam_range[0]) & (lam < lam_range[1])
lam=lam[num]
galaxy=galaxy[num]
# galaxy1=galaxy1[num]
# galaxy_emm=galaxy_emm[num]
galaxy1=galaxy1_mock[num]
galaxy0=galaxy0_mock[num]
noise=noise[num]
mask=mask[num]


In [ ]:
"""
input weights and attenuated galaxy spectrum, initial galaxy spectrum, attenuation curve
"""
# sps_ppxf = ppxf_lib.sps_lib(temp_file, velscale, FWHM_gal, wave_range=[3000,10000])
temp_file=os.path.join(os.getcwd(),'sps_models/BC03_SSP_tpl_chab.fits')
sps_bc03 = util.temp(temp_file,velscale=velscale,FWHM_gal=2.76,FWHM_tem=2.5,normalize=True)
plt.figure(figsize=(20,5))
ax1=plt.axes([0.05,0.1,0.3,0.8])
n_star=120
n_age=sps_bc03.templates.shape[1]
n_metal=sps_bc03.templates.shape[2]
sps_bc03.plot(weights[:n_star].reshape(n_age,n_metal))
ax2=plt.axes([0.38,0.1,0.6,0.8])
plt.plot(lam,galaxy,'k',label='input att galaxy')
# plt.plot(lam,galaxy0,'r',label='input att galaxy')
# plt.plot(lam_mock,galaxy1_mock,label='input init galaxy')
# plt.plot(lam,galaxy,'--',label='input att galaxy')
plt.plot(lam_mock,galaxy0_mock,label='input init galaxy')
plt.plot(lam,galaxy1,label='input init galaxy')
# plt.plot(lam,galaxy0*att_fun(lam,ebv,1.3)*2.6)
# plt.plot(lam_temp,(templates[:,comp==0]@weights[comp==0]*att_fun(lam_temp,ebv,1.3))+(templates[:,comp!=0]@weights[comp!=0]),'--',label='att galaxy')
# plt.plot(lam_temp0,(temp0[:,comp==0]@weights[comp==0]*att_fun(lam_temp0,ebv,1.3))+(temp0[:,comp!=0]@weights[comp!=0]),'r--',label='att galaxy')
plt.xlabel(r'$\lambda(\AA)$',fontsize=10)
# plt.xlim(3600,4500)
plt.legend(fontsize=15)


In [ ]:
from ppxf.ppxf import ppxf as pp
ext = lambda x,delta:(x/5500)**-delta


def att_fun1(lam,ebv,delta,f_nodust=0):
    # delta=-np.log(1+1/Rv)/np.log(89/110)
    
    Rv=1/(((89/110)**-delta)-1)
    # k=(110.-89.**2/110./Rv)/21.
    # ext = lambda x:k*(x/5500)**-1+(1-k)*(x/5500)**-2
    frac = 10**(-0.4*ebv*Rv*ext(lam,delta).clip(0))
    frac = f_nodust + (1-f_nodust)*frac
    return frac

start = [0, 180.]
# params =np.array([50, 80, -10, 100, 30, 50])
n_temps = temp.shape[1] 
n_forbidden = np.sum(["[" in a or "HeI" in a for a in gas_names])
n_balmer = len(gas_names) - n_forbidden
component = np.array([0]*n_temps + [1]*n_balmer + [2]*n_forbidden)
if AGN>0:
    component = np.insert(component,120,0)
gas_component = np.array(component) >0
moments = [2,2,2]
start = [start, start[:2], start[:2]]


dust2= [{"start": [0.5,1.3], "bounds": [[0, 2], [0.5,1.5]], "func":att_fun1, "component": ~gas_component,"fixed":[False,False]}]
dust3= [{"start": [0.5], "bounds": [[0, 2]], "component": ~gas_component,"fixed":[False]}]



In [ ]:
'''
ewsps
'''

%time p1 = ewfit(templates, galaxy, noise, velscale, start,Z=Z, moments=moments ,degree=-1,dust=None,linear_method='lsq_box',plot=False,clean=False, lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names, tie_balmer= tie_balmer, limit_doublets=limit_doublets,l=1e-8)

In [ ]:
p1.plot()

In [ ]:
'''
ppxf+power attenuation curve
'''
%time p2=pp(templates, galaxy, noise, velscale, start, moments=moments, degree=-1, dust=dust2, linear_method='lsq_box', plot=False, clean=False, lam=lam, lam_temp=lam_temp, component=component, gas_component=gas_component, gas_names=gas_names,quiet=False)

In [ ]:
p2.plot()

In [ ]:
'''
ppxf+calzetti curve
'''
p3=pp(templates, galaxy, noise, velscale, start, moments=moments, degree=-1 ,dust=dust3,linear_method='lsq_box',plot=False,clean=False,lam=lam, lam_temp=lam_temp, component=component,gas_component=gas_component, gas_names=gas_names)

In [ ]:
sps_bc03.mean_age_metal(weights[:n_star].reshape(n_age,n_metal))
sps_bc03.mean_age_metal(p1.weights[:n_temps].reshape(n_age,n_metal))
sps_bc03.mean_age_metal(p2.weights[:n_temps].reshape(n_age,n_metal))
sps_bc03.mean_age_metal(p3.weights[:n_temps].reshape(n_age,n_metal))


In [ ]:
plt.figure(figsize=(20,20))
plt.axes([0.05,0.78,0.23,0.2])
# n_age=sps_bc03.templates.shape[1]
# n_metal=sps_bc03.templates.shape[2]
n_age=sps_bc03.templates.shape[1]
n_metal=sps_bc03.templates.shape[2]
sps_bc03.plot(weights[:n_temps].reshape(n_age,n_metal),title='input weights')


plt.axes([0.05,0.53,0.23,0.2])
sps_bc03.plot(p1.weights[:n_temps].reshape(n_age,n_metal),title='weights of ewfit')
# plt.subplot(343)
# sps_bc03.plot(p1t.weights[:n_temps].reshape(n_age,n_metal))
plt.axes([0.05,0.28,0.23,0.2])
sps_bc03.plot(p2.weights[:n_temps].reshape(n_age,n_metal)/np.sum(p2.weights[:n_temps]),title='weights of ppxf+powerlaw')
plt.axes([0.05,0.03,0.23,0.2])
sps_bc03.plot(p3.weights[:n_temps].reshape(n_age,n_metal)/np.sum(p3.weights[:n_temps]),title='weights of ppxf+calzetti')

plt.axes([0.325,0.78,0.66,0.2])
plt.title('input spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',label = 'mock initial spectrum')
plt.plot(p1.lam,p1.galaxy,'grey', label = 'input spectrum')
plt.plot(p1.lam,p1.galaxy_l, 'dodgerblue', label = 'smoothed spectrum')
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux')
plt.xlim(3500,9000)
plt.legend(fontsize=15)

plt.axes([0.325,0.53,0.66,0.2])
plt.title('EW process and fit')
plt.plot(p1.lam,p1.galaxy/p1.galaxy_l,'grey',label='EW-processed spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])+(p1.gal_temp[:,p1.degree+1:][:,p1.gas_component]@weights[p1.gas_component])/p1.galaxy_l,'k',label='initial EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component]),'k--')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])+(p1.gal_temp[:,p1.degree+1:][:,gas_component]@p1.weights[gas_component])/p1.galaxy_l,'dodgerblue',ls='--',label='fit EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component]),'dodgerblue',ls='-.')
plt.legend(fontsize=15)
plt.plot([3700,3750],[1.5,1.67],'k')
plt.plot([4150,6500],[1.5,1.67],'k')
plt.plot([3700,4150,4150,3700,3700],[0.5,0.5,1.5,1.5,0.5],'k')
plt.xlim(3500,9000)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux')

ax=plt.axes([0.355,0.615,0.33,0.11])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy/p1.galaxy_l,'grey',label='EW-processed spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])+(p1.gal_temp[:,p1.degree+1:][:,p1.gas_component]@weights[p1.gas_component])/p1.galaxy_l,'k',label='initial EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~p1.gas_component]@weights[~p1.gas_component]),'k--')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])+(p1.gal_temp[:,p1.degree+1:][:,gas_component]@p1.weights[gas_component])/p1.galaxy_l,'dodgerblue',ls='--',label='fit EW spectrum')
plt.plot(p1.lam,(p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component])/(p1.gal_temp_l[:,p1.degree+1:][:,~gas_component]@p1.weights[~gas_component]),'dodgerblue',ls='-.')
plt.xlim(3700,4150)
plt.ylim(0.5,1.5)

plt.axes([0.325,0.28,0.66,0.2])
plt.title('full spectrum')
plt.plot(p1.lam,p1.galaxy,'grey',lw=3, label = 'input spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=3, label = 'mock initial spectrum')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:]@p1.weights,'dodgerblue',ls='--',lw=2, dashes=(10,5),label='ewfit spectrum')
# plt.plot(p1.lam,p1t.gal_temp@p1t.weights,'grey',label='ewfit')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'r--',lw=2, dashes=(3,5),label='ppxf+powerlaw')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'g--',lw=2,label='ppxf+calzetti')
plt.legend(fontsize=15)
plt.plot([3700,3750],[2,2.2],'k')
plt.plot([4150,6500],[2,2.2],'k')
plt.plot([3700,4150,4150,3700,3700],[0.5,0.5,2,2,0.5],'k')
plt.xlim(3500,9000)
plt.ylim(0.3,4.78)
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel('normed flux')

ax=plt.axes([0.355,0.365,0.33,0.11])
ax.patch.set_alpha(0)
plt.plot(p1.lam,p1.galaxy,'grey',lw=5, label = 'input spectrum')
plt.plot(lam_mock,galaxy0_mock,'k',lw=5, label = 'mock initial spectrum')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:]@p1.weights,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='ewfit spectrum')

# plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:]@p1t.weights,'dodgerblue',ls='--',lw=3, dashes=(10,5),label='ewfit spectrum')
# plt.plot(p1t.lam,p1.gal_temp[:,p1.degree+1:][:,:120]@weights[:120],'k',lw=3, dashes=(10,5),label='ewfit spectrum')
# plt.plot(p1.lam,p1.gal_temp[:,:120]@p1.weights[:120],'dodgerblue',ls='--',lw=3, dashes=(10,5),label='ewfit spectrum')
# plt.plot(p1.lam,p1.gal_temp@p1t.weights,'grey',label='ewfit')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p2.weights[~gas_component]/np.sum(p2.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'r--',lw=3, dashes=(3,5),label='ppxf+powerlaw')
plt.plot(p1.lam,p1.gal_temp[:,p1.degree+1:][:,~gas_component]@p3.weights[~gas_component]/np.sum(p3.weights[~gas_component])+p1.gal_temp[:,p1.degree+1:][:,gas_component]@p2.weights[gas_component],'g--',lw=3,label='ppxf+calzetti')
plt.xlim(3700,4150)
plt.ylim(0.5,2)


plt.axes([0.325,0.03,0.66,0.2])
plt.title('attenuation curve')
# plt.plot(p1.lam, p1.att_curve,'b.',label='ewfit',alpha=0.3)
plt.plot(p1.lam, p1.att_curve,'b.',label='ewfit',alpha=0.3)
ebv=0.3
ext = lambda x,delta:(x/5500)**-delta
att = lambda x, ebv, delta: ebv*(ext(x,delta)-1)/(((89/110)**-delta)-1)
# plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k',lw=5,label='mock')
plt.plot(p1.lam,att(p1.lam,ebv,1.3),'k--',lw=5,label='mock attenuaiton')
# plt.plot(p1.lam,att(p1.lam,ebv,1.3)+resp(p1.lam),'k',lw=5,label='mock attenuaiton+bias')
# plt.plot(p1.lam,resp(p1.lam),'k-.',lw=5,label='bias')
# plt.plot(p1.lam,smooth(att(p1.lam,ebv,1.3)+2.5*(resp(p1.lam)),p1.lam,np.full_like(p1.lam,1),1e5,0.5)[0],'grey',ls='--',lw=5,label='mock attenuaiton+bias')

plt.plot(p1.lam, p1.att_curve_smooth,'dodgerblue',ls='--',lw=3,label='smoothed ewfit')
# plt.plot(lam,2.5*np.log10((galaxy0-galaxy_emm)/(galaxy1-galaxy_emm)))
plt.plot(p2.lam,-2.5*np.log10(p2.dust[0]['func'](p2.lam,*p2.dust[0]['sol'])/p2.dust[0]['func'](5500,*p2.dust[0]['sol'])),'r--',lw=3,label='PpXF+powerlaw')
plt.plot(p3.lam,-2.5*np.log10(p3.dust[0]['func'](p3.lam,*p3.dust[0]['sol'])/p3.dust[0]['func'](5500,*p3.dust[0]['sol'])),'g--',lw=3,label='PpXF+calzetti')
plt.xlabel(r'$\lambda(\AA)$')
plt.ylabel(r'$A_\lambda-A_V$')
plt.xlim(3550,9000)
plt.ylim(-0.7,1.)
plt.legend(ncol=2,fontsize=15)

plt.tight_layout()